In [2]:
import openai
import os
import pandas as pd

openai.api_key = os.getenv("OPENAI_API_KEY")

## Step 1: Get summaries and tags for each post

The first step is to upload a CSV with two columns named "link" and "title".

In [3]:
df = pd.read_csv("./sheets/all-alignment-forum-posts-2025-filtered.csv")
df.head()

,title,link,karma,date,keep
0,Eliciting secret knowledge from language models,https://www.alignmentforum.org/posts/Mv3yg7wMX...,20,2025-10-02T20:57:13.496Z,y
1,Lectures on statistical learning theory for al...,https://www.alignmentforum.org/posts/yAwnYoeCz...,15,2025-10-01T08:36:52.525Z,NaN
2,AI Induced Psychosis: A shallow investigation,https://www.alignmentforum.org/posts/iGF7YcnQk...,83,2025-08-26T20:03:53.308Z,NaN
3,Stress Testing Deliberative Alignment for Anti...,https://www.alignmentforum.org/posts/JmRfgNYCr...,57,2025-09-17T16:59:12.906Z,y
4,Four ways learning Econ makes people dumber re...,https://www.alignmentforum.org/posts/xJWBofhLQ...,119,2025-08-21T17:52:46.684Z,NaN


Convert the date column to a datetime object and then to a formatted date.

In [4]:
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
df.head()

,title,link,karma,date,keep
0,Eliciting secret knowledge from language models,https://www.alignmentforum.org/posts/Mv3yg7wMX...,20,2025-10-02,y
1,Lectures on statistical learning theory for al...,https://www.alignmentforum.org/posts/yAwnYoeCz...,15,2025-10-01,NaN
2,AI Induced Psychosis: A shallow investigation,https://www.alignmentforum.org/posts/iGF7YcnQk...,83,2025-08-26,NaN
3,Stress Testing Deliberative Alignment for Anti...,https://www.alignmentforum.org/posts/JmRfgNYCr...,57,2025-09-17,y
4,Four ways learning Econ makes people dumber re...,https://www.alignmentforum.org/posts/xJWBofhLQ...,119,2025-08-21,NaN


In [5]:
# Filter to only keep rows where the 'keep' column has value 'y'
df_filtered = df[df["keep"] == "y"].copy()

# Reset the index to have continuous numbering
df_filtered = df_filtered.reset_index(drop=True)

print(f"Original dataframe: {len(df)} rows")
print(f"Filtered dataframe: {len(df_filtered)} rows")
print(f"\nFiltered dataframe preview:")
df_filtered.head()

Original dataframe: 438 rows
Filtered dataframe: 98 rows

Filtered dataframe preview:


,title,link,karma,date,keep
0,Eliciting secret knowledge from language models,https://www.alignmentforum.org/posts/Mv3yg7wMX...,20,2025-10-02,y
1,Stress Testing Deliberative Alignment for Anti...,https://www.alignmentforum.org/posts/JmRfgNYCr...,57,2025-09-17,y
2,Research Agenda: Synthesizing Standalone World...,https://www.alignmentforum.org/posts/LngR93Ywi...,35,2025-09-22,y
3,"What, if not agency?",https://www.alignmentforum.org/posts/tQ9vWm4b5...,40,2025-09-15,y
4,Subliminal Learning: LLMs Transmit Behavioral ...,https://www.alignmentforum.org/posts/cGcwQDKAK...,105,2025-07-22,y


In [6]:
df_filtered.to_csv("output/alignment-forum-posts-2025-curated.csv", index=False)

Then use Python to open each link and get the text. Give the text to an LLM and ask it write a summary.

For each link, create a summary and add it to a new column called "summary".

In [64]:
summary_prompt = """
You are an academic researcher and your task is to help write a literature review of the field of AI alignment.

Your current task is to read a blog post from the AI Alignment Forum and summarize it in 100 words.

You should also return a list of tags or keywords that best describe the blog post. You should return a list of 1-5 tags for each blog post.
"""

In [83]:
import requests
from bs4 import BeautifulSoup
import time
from openai import OpenAI
from pydantic import BaseModel
import random

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def get_webpage_content(url, title):
    time.sleep(random.uniform(0.5, 1.5))
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            print(f"Successfully fetched webpage content for {title}")
            soup = BeautifulSoup(response.text, "html.parser")
            webpage_text = soup.get_text()
            webpage_text = f"title: {title}, content: {webpage_text}"
            return webpage_text
        else:
            return f"Error: Unable to access page (Status code: {response.status_code})"
    except Exception as e:
        return "Error: Unable to access page"

client = OpenAI()

class Summary(BaseModel):
    summary: str
    tags: list[str]

def call_openai_api(content, response_format, system_prompt, user_prompt_template=None):
    """
    Call OpenAI API with structured output parsing.

    Args:
        content: The content to be processed
        response_format: The Pydantic model class for response parsing
        system_prompt: The system message/instructions
        user_prompt_template: Optional template for user message.
                             Should include {content} placeholder.
                             Defaults to "Here is the blog post content: {content}"
    """
    try:
        # Use default user prompt template if none provided
        if user_prompt_template is None:
            user_prompt_template = "Here is the blog post content: {content}"

        # Format the user prompt with the content
        user_message = user_prompt_template.format(content=content)

        response = client.responses.parse(
            model="gpt-5",
            input=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message},
            ],
            reasoning={
                "effort": "minimal",
            },
            text_format=response_format,
        )
        print("Successfully called OpenAI API")

        parsed_response = response.output_parsed
        return parsed_response
    except Exception as e:
        print("Error when calling OpenAI API:", e)
        return None

def get_summary(link, title):
    webpage_content = get_webpage_content(link, title)
    response_obj = call_openai_api(
        webpage_content,
        Summary,
        summary_prompt,
    )
    if response_obj is None:
        return None, None
    print(f"Successfully got summary and tags for {title}", end="\n\n")
    ai_summary = response_obj.summary
    ai_tags = response_obj.tags
    return ai_summary, ai_tags


In [66]:
df_filtered.head()

,title,link,karma,date,keep
0,Eliciting secret knowledge from language models,https://www.alignmentforum.org/posts/Mv3yg7wMX...,20,2025-10-02,y
1,Stress Testing Deliberative Alignment for Anti...,https://www.alignmentforum.org/posts/JmRfgNYCr...,57,2025-09-17,y
2,Research Agenda: Synthesizing Standalone World...,https://www.alignmentforum.org/posts/LngR93Ywi...,35,2025-09-22,y
3,"What, if not agency?",https://www.alignmentforum.org/posts/tQ9vWm4b5...,40,2025-09-15,y
4,Subliminal Learning: LLMs Transmit Behavioral ...,https://www.alignmentforum.org/posts/cGcwQDKAK...,105,2025-07-22,y


In [67]:
size = df_filtered.shape[0]

summaries = [''] * size
tags = [''] * size

i = 0
for index, row in df_filtered.iterrows():
    # if i >= 2:
    #     break
    print(f'Processing row {index + 1} of {df_filtered.shape[0]}')
    summary, tag = get_summary(row["link"], row["title"])
    summaries[index] = summary
    tags[index] = tag
    i += 1

df_filtered["summary"] = summaries
df_filtered["tags"] = tags

Processing row 1 of 98
Successfully fetched webpage content for Eliciting secret knowledge from language models
Successfully called OpenAI API
Successfully got summary and tags for Eliciting secret knowledge from language models

Processing row 2 of 98
Successfully fetched webpage content for Stress Testing Deliberative Alignment for Anti-Scheming Training
Successfully called OpenAI API
Successfully got summary and tags for Stress Testing Deliberative Alignment for Anti-Scheming Training

Processing row 3 of 98
Successfully fetched webpage content for Research Agenda: Synthesizing Standalone World-Models
Successfully called OpenAI API
Successfully got summary and tags for Research Agenda: Synthesizing Standalone World-Models

Processing row 4 of 98
Successfully fetched webpage content for What, if not agency?
Successfully called OpenAI API
Successfully got summary and tags for What, if not agency?

Processing row 5 of 98
Successfully fetched webpage content for Subliminal Learning: LLM

In [68]:
df_filtered.head()

,title,link,karma,date,keep,summary,tags
0,Eliciting secret knowledge from language models,https://www.alignmentforum.org/posts/Mv3yg7wMX...,20,2025-10-02,y,The post presents a benchmark for eliciting “s...,"[AI alignment, Model honesty and deception, El..."
1,Stress Testing Deliberative Alignment for Anti...,https://www.alignmentforum.org/posts/JmRfgNYCr...,57,2025-09-17,y,Linkpost summarizing a collaboration (OpenAI +...,"[Anti-scheming, Covert actions, Situational aw..."
2,Research Agenda: Synthesizing Standalone World...,https://www.alignmentforum.org/posts/LngR93Ywi...,35,2025-09-22,y,Thane Ruthenis proposes an alignment agenda to...,"[AI alignment, World-models, Natural abstracti..."
3,"What, if not agency?",https://www.alignmentforum.org/posts/tQ9vWm4b5...,40,2025-09-15,y,Abram Demski summarizes and partially steelman...,"[Co-agency vs agency, Soloware and interfaces,..."
4,Subliminal Learning: LLMs Transmit Behavioral ...,https://www.alignmentforum.org/posts/cGcwQDKAK...,105,2025-07-22,y,"The authors identify “subliminal learning,” wh...","[distillation, safety and alignment, model-gen..."


In [69]:
df_filtered.to_csv("output/summaries-and-tags.csv", index=False)

## Step 2: Convert the CSV containing the summaries and tags to a document

In [70]:
df = pd.read_csv("output/summaries-and-tags.csv")
df.head()

,title,link,karma,date,keep,summary,tags
0,Eliciting secret knowledge from language models,https://www.alignmentforum.org/posts/Mv3yg7wMX...,20,2025-10-02,y,The post presents a benchmark for eliciting “s...,"['AI alignment', 'Model honesty and deception'..."
1,Stress Testing Deliberative Alignment for Anti...,https://www.alignmentforum.org/posts/JmRfgNYCr...,57,2025-09-17,y,Linkpost summarizing a collaboration (OpenAI +...,"['Anti-scheming', 'Covert actions', 'Situation..."
2,Research Agenda: Synthesizing Standalone World...,https://www.alignmentforum.org/posts/LngR93Ywi...,35,2025-09-22,y,Thane Ruthenis proposes an alignment agenda to...,"['AI alignment', 'World-models', 'Natural abst..."
3,"What, if not agency?",https://www.alignmentforum.org/posts/tQ9vWm4b5...,40,2025-09-15,y,Abram Demski summarizes and partially steelman...,"['Co-agency vs agency', 'Soloware and interfac..."
4,Subliminal Learning: LLMs Transmit Behavioral ...,https://www.alignmentforum.org/posts/cGcwQDKAK...,105,2025-07-22,y,"The authors identify “subliminal learning,” wh...","['distillation', 'safety and alignment', 'mode..."


In [78]:
import ast

def df_to_document(df):
    """
    Convert the dataframe to string representation with the following format for each post:
    title: <title>
    summary: <summary>
    tags: <tags>
    """
    row_strings = []
    for index, row in df.iterrows():
        summary_text = row['summary']
        tags_list = ast.literal_eval(row['tags'])
        tags_text = ', '.join(tags_list)

        title_string = f"## [{row['title']}]({row['link']})"
        # post_string = f"{title_string}\n- **karma:** {row['karma']}\n- **date:** {row['date']}\n- **summary:** {summary_text}\n- **tags:** {tags_text}\n"
        post_string = f"{title_string}\n- **summary:** {summary_text}\n- **tags:** {tags_text}\n"
        row_strings.append(post_string)
    return "\n".join(row_strings)

document = df_to_document(df)
print(document)

with open("output/summaries-and-tags.md", "w") as f:
    f.write(document)

## [Eliciting secret knowledge from language models](https://www.alignmentforum.org/posts/Mv3yg7wMXfns3NPaz/eliciting-secret-knowledge-from-language-models-1)
- **summary:** The post presents a benchmark for eliciting “secret knowledge” that language models possess but deny. The authors fine-tune LLMs to: (1) know a secret word, (2) follow a hidden instruction (Base64-encoded), and (3) infer a user’s gender—while refusing to admit this knowledge. They evaluate black-box elicitation (adversarial prompts, prefill attacks, persona sampling, many-shot jailbreaking) and white-box methods using mechanistic interpretability (logit lens, sparse autoencoders). In two of three settings, black-box prefill and persona-sampling work best; in the remaining setting, white-box methods perform better. All methods beat simple baselines. They release models and code to seed further auditing research.
- **tags:** AI alignment, Model honesty and deception, Elicitation/Auditing, Mechanistic interpretability

## Step 3: Feed summaries and tags to an LLM and ask it to write a literature review.

Now that we have a 100-word summary and list of 1-5 tags for each post, we can feed all these summaries and tags to an LLM and ask it to write a literature review.

We will instruct the LLM to read all the summaries and tags and describe the field of AI alignment today or create a taxonomy of the field for us.

In [79]:
class Taxonomy(BaseModel):
    taxonomy: str
    editorial: str

taxonomy_prompt = """
You are an academic research and your task is to help write a literature review of the field of AI alignment.

You will be given a list of blog posts to read about AI alignment. Each row consists of a blog post title, a 100-word summary, and a list of 1-10 tags that best describe the blog post separated by a new line character
and each row is separated by a double new line character.

Your first task is to create a taxonomy of the field of AI alignment which you should simply return as a bullet list of topics. You should use the tags to help you create the taxonomy.

Your second task is to write a 500-word editorial describing the current landscape of the field of AI alignment in 2024. You should use the summaries and tags to help you write the editorial.
"""

In [86]:
def row_to_string(row):
    s = f"title: {row['title']}\nsummary: {row['summary']}\ntags: {row['tags']}\n\n"
    return s

def df_to_string(df):
    """
    Convert the dataframe to string representation with the following format for each row:
    title: <title>
    summary: <summary>
    tags: <tags>
    """
    row_strings = []
    for index, row in df.iterrows():
        row_strings.append(row_to_string(row))
    return "\n".join(row_strings)

df_string = df_to_string(df)
df_string_words = len(df_string.split())
print(f"Number of words in the dataframe string: {df_string_words:,}")

Number of words in the dataframe string: 12,145


In [84]:
taxonomy_response_obj = call_openai_api(
    content=df_string,
    response_format=Taxonomy,
    system_prompt=taxonomy_prompt,
    user_prompt_template="Here is the list of blog post rows where each row has a title, a 100-word summary, and a list of 1-10 tags: {content}"
)
taxonomy = taxonomy_response_obj.taxonomy
editorial = taxonomy_response_obj.editorial

Successfully called OpenAI API


In [87]:
print(taxonomy)

- Alignment problem definitions and philosophy
- AI control, monitoring, and oversight protocols
- Deception, scheming, and sandbagging
- Chain-of-thought: faithfulness, monitoring, and process vs outcome supervision
- Emergent misalignment and fine-tuning safety
- Interpretability: mechanistic methods, SAEs, representation engineering, model diffing, and weight/parameter decomposition
- World-models, abstractions, and natural latents
- Learning theory foundations: SLT↔AIT, infra-Bayesianism, OOD generalization
- Post-training, elicitation, unlearning, and distillation
- Reward design, behaviorist vs non-behaviorist RL, myopia
- Evaluations, benchmarks, and threat models
- Agent foundations, decision theory, and corrigibility
- Governance, security, and timelines/takeoff dynamics
- Agentic systems and autonomous agents
- Funding initiatives, agendas, and community infrastructure


In [88]:
print(editorial)

AI alignment in 2024–2025 is consolidating into two intertwined thrusts: practical control of increasingly capable LLM-based agents, and deeper scientific understanding of learning, representation, and deception. Multiple lines of evidence show today’s systems can both help and hinder safety work, creating a narrow window to build monitoring and control infrastructure while probing fundamental failure modes.

Control and oversight are maturing rapidly. Labs and evaluators (METR, UK AISI, Anthropic, OpenAI) advance time-horizon metrics, red-team testbeds, and agent-control protocols (resampling and auditing). Chain-of-thought (CoT) monitoring emerges as a high-leverage but fragile opportunity: several posts find that readable traces greatly aid detection of subtle sabotage and misbehavior, and inference-time-compute tends to increase CoT faithfulness. Yet unfaithful reasoning can fool monitors, and strong optimization against CoT may push models toward neuralese. Strategy documents urge